In [1]:
import comet_ml
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torchinfo import summary
from comet_ml.integration.pytorch import log_model

In [2]:
from skibidi_face_detector.dataset.small_celebrities import test_loader as small_test_loader, train_loader as small_train_loader
from skibidi_face_detector.dataset.celeba import val_num_classes, val_train_loader, val_test_loader, augment, transformer, BATCH_SIZE
from skibidi_face_detector.face_embedder.Model import Model

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [3]:
PROJECT_NAME = 'dlf-train'
EXPERIMENT_NAME = 'extraordinary_bedi'

In [4]:
model = Model(
    val_num_classes,
    arc_face_margin=0.2,
    embedding_dim=128,
    freeze_feature_extractor=True,
    hidden_layer_features=2048,
    learning_rate=0.0005,
    p_dropout_1=0.2,
    p_dropout_2=0.2,
    scale=30,
    triplet_margin=0.2,
    augments=augment,
    transformer=transformer,
    arc_face_loss_multiplier=0.0,
    triplet_loss_multiplier=1.0,
    accuracy_loaders=(small_train_loader, small_test_loader)
)

In [5]:
comet_logger = CometLogger(
    save_dir="./",
    project_name=f"{PROJECT_NAME}",
    experiment_name=f"{EXPERIMENT_NAME}",
)

CometLogger will be initialized in online mode


In [6]:
comet_logger.experiment.set_model_graph(
        f"{model.__repr__()}\n"
        f"{summary(model, input_size=(BATCH_SIZE, 3, 224, 224), depth=9)}",
        overwrite=True,
    )

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/superrmurlocc/dlf-train/0e05261cfcbb476aae340120200021a9



In [7]:
comet_logger.experiment.log_code('skibidi_face_detector/face_embedder/ArcFaceLoss.py')
comet_logger.experiment.log_code('skibidi_face_detector/face_embedder/TripletLoss.py')
comet_logger.experiment.log_code('skibidi_face_detector/face_embedder/Embedder.py')
comet_logger.experiment.log_code('skibidi_face_detector/face_embedder/Model.py')
comet_logger.experiment.log_code('skibidi_face_detector/face_embedder/VggFace2.py')

[('skibidi_face_detector/face_embedder/VggFace2.py',
  {'web': 'https://www.comet.com/api/asset/download?assetId=10648893d2a342bd8d471c85c5bc169a&experimentKey=0e05261cfcbb476aae340120200021a9',
   'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=10648893d2a342bd8d471c85c5bc169a&experimentKey=0e05261cfcbb476aae340120200021a9',
   'assetId': '10648893d2a342bd8d471c85c5bc169a'})]

In [8]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_accuracy',
    dirpath=f'checkpoints/{EXPERIMENT_NAME}/',
    filename='best-{epoch:02d}-{val_accuracy:.2f}',
    save_top_k=1,
    mode='min'
)

latest_checkpoint_callback = ModelCheckpoint(
    dirpath=f'checkpoints/{EXPERIMENT_NAME}/',
    filename='latest-{epoch:02d}',
    save_top_k=1,
    every_n_epochs=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=50,
    mode='min'
)

In [9]:
trainer = pl.Trainer(
    max_epochs=10,
    callbacks=[checkpoint_callback, early_stopping_callback, latest_checkpoint_callback],
    logger=comet_logger
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model, val_train_loader, val_test_loader)

/Users/jakubbednarski/Desktop/Studia/sem2/dlf/venv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name          | Type        | Params | Mode 
------------------------------------------------------
0 | embedder      | Embedder    | 30.0 M | train
1 | arc_face_loss | ArcFaceLoss | 2.5 M  | train
2 | triplet_loss  | TripletLoss | 0      | train
3 | augments      | Compose     | 0      | train
4 | transformer   | Compose     | 0      | train
------------------------------------------------------
7.0 M     Trainable params
25.6 M    Non-trainable params
32.6 M    Total params
130.235   Total estimated model params size (MB)
164       Modules in train mode
0         Modules in eval mode
COMET WARNING: Set model graph ignored; already called. Call with overwrite=True to replace graph definition


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

/Users/jakubbednarski/Desktop/Studia/sem2/dlf/venv/lib/python3.11/site-packages/pytorch_lightning/loops/optimization/automatic.py:132: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
COMET WARNING: Failed to log system metrics: [sys.ram,sys.cpu,sys.load]
COMET WARNING: Failed to log system metrics: [sys.ram,sys.cpu,sys.load]
COMET WARNING: Failed to log system metrics: [sys.ram,sys.cpu,sys.load]
COMET WARNING: Failed to log system metrics: [sys.ram,sys.cpu,sys.load]
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : extraordinary_bedi
COMET INFO:     url                   : https://www.comet.com/superrmurlocc/dlf-train/0e05261cfcbb476aae340120200021a9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [268]                     : (0.0, 18.5624942779541)
COMET INFO:     train_arc_face_loss_epoch [10] : (19.177770614624023,

In [11]:
model_name = f"{type(model).__name__}_{comet_logger.experiment.get_key()}"
log_model(comet_logger.experiment, model, model_name=model_name)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/superrmurlocc/dlf-train/0e05261cfcbb476aae340120200021a9



In [12]:
comet_logger.experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : extraordinary_bedi
COMET INFO:     url                   : https://www.comet.com/superrmurlocc/dlf-train/0e05261cfcbb476aae340120200021a9
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name         : extraordinary_bedi
COMET INFO:   Uploads:
COMET INFO:     model-element : 2 (124.51 MB)
COMET INFO: 
COMET INFO: Please wait for assets to finish uploading (timeout is 10800 seconds)
COMET INFO: Still uploading 1 file(s), remaining 124.51 MB/124.51 MB
COMET INFO: Still uploading 1 asset(s), remaining 24.51 MB/124.51 MB, Throughput 6.63 MB/s, ETA ~4s
